In [2]:
import numpy as np
import os

from __future__ import unicode_literals
import glob, os, sys,re
import numpy as np
import spacy
from random import shuffle

reload(sys)
sys.setdefaultencoding('utf8')

nlp = spacy.load('en')

root_dir = '/data/corpora/ted/transcripts_clean/'

In [3]:
applause_tags = [
    'Applause',
    '(Applause)',
    'APPLAUSE AND CHEERS',
    'Applause.',
    'CHEERING & APPLAUSE',
    'CHEERING AND APPLAUSE',
    'cheers and applause continuing',
    'Cheers, applause, chanting',
    'Cheers, applause',
    'Cheers, chanting, applause',
    'continued applause',
    'extended cheers, applause',
    'inaudible, cheers, applause',
    'laughter, applause',
    'Laughter, applause, cheers',
    'laughter, cheers, applause',
    'mixed cheers and boos, applause',
    'scattered applause',
    'scattered cheers, applause',
    'Sustained applause',
    'Sustained cheers and applause',
    'Sustained cheers, applause',
    'applause.',
    'applause and cheering',
    'applpause',
    'cheering and applause',
    'cheers and applause',
    'cheers, laughter, and applause',
    'extended cheers and applause',
    'laughter and applause.',
    'laughter and claps.',
    'laughter, cheers, and applause',
    'pplause.',
    'scattered applause.',
    'sustained applause.'
]

In [4]:
transcript_files = [root_dir + f for f in os.listdir(root_dir)]
missing = []
for t in transcript_files:
    text = open(t).read()
    if text == '':
        missing.append(t)
transcript_files = [t for t in transcript_files if t not in missing]

In [5]:
import os

pathname="/data/corpora/ted/transcripts_clean/"
files = [pathname + f for f in os.listdir(pathname)]
talk_names = open('/home/jrgillick/Applause/talk_names.txt').read().replace('.txt','.html').split('\n')
talk_names = ['isabel_allende_how_to_live_passionately_no_matter_your_age.html' if t == 'isabelle_allende_how_to_live_passionately_no_matter_your_age.html' else t for t in talk_names
]
talk_names = [t for t in talk_names if t not in ['', 'test.html', '\r']]
transcript_files = [pathname + t for t in talk_names]
len(transcript_files)

904

In [9]:
tag_hash = {}
for tag in applause_tags:
    tag_hash[tag] = []
    
for index, f in enumerate(transcript_files):
    text = open(f).read()
    for tag in tag_hash:
        if tag in text:
            tag_hash[tag].append(index)
            
used_tags = [tag for tag in applause_tags if len(tag_hash[tag]) > 0]
used_tags

[u'Applause',
 u'(Applause)',
 u'Applause.',
 u'applause.',
 u'applause and cheering',
 u'pplause.']

In [13]:
class Transcript:
    def __init__(self,transcript_file):
        self.text = open(transcript_file).read()
        self.lines = self.get_lines()
        self.word_list = self.get_word_list()        
        
    def get_word_list(self):
        return self.text.replace('\n',' ').replace('\t',' ').split(' ')
    
    def get_lines(self):
        lines = self.text.split('\n')
        lines = [l for l in lines if len(l) > 0]
        split_lines = [line.split('\t') for line in lines]
        return split_lines
        
    def get_timestamps(self):
        return [l[0] for l in self.lines]
    
    def get_text_lines(self):
        return [l[1] for l in self.lines]
    
    def count_words_in_line(self,line):
        return len(line.split(' '))
        
    def count_words_following(self,line_index, word_index):
        future_lines = self.get_text_lines()[line_index+1:]
        current_line_words = self.count_words_in_line(self.get_text_lines()[line_index]) - word_index - 1
        total_future_line_words = sum([self.count_words_in_line(l) for l in future_lines])
        return current_line_words + total_future_line_words
    
    def get_word(self, line_index, word_index):
        lines = self.get_text_lines()
        return lines[line_index].split(' ')[word_index]
    
    def get_line(self, line_index):
        return self.get_text_lines()[line_index]
    
    def get_preceding_line(self, line_index, word_index):
        lines = self.get_text_lines()
        temp_index = line_index
        if word_index == 0:
            line = ""
            word_len = 0;
        else:
            line = self.get_text_lines()[line_index]
            words = line.split(' ')
            line = ' '.join(words[0:word_index])
            word_len = word_index
        while word_len < 10 and temp_index > 0:
            temp_index -= 1
            line = lines[temp_index] + " " + line
            word_len = len(line.split(" "))
        return line
        
            #return ' '.join(words[0:word_index])
    
    def get_word_index_of_applause_in_line(self,line):
        words = line.split(' ')
        for index, word in enumerate(words):
            if '(Applause)' in word:
                return index
        return -999
    
    # Returns (line_index, word_index)
    def find_applause_instances2(self, exclude_final = False):
        lines = self.get_text_lines()
        instances = []
        for line_index, l in enumerate(lines):
            for tag in applause_tags:
                if tag in l:
                    word_index = self.get_word_index_of_applause_in_line(l)
                    if exclude_final:
                        words_following = self.count_words_following(line_index,word_index)
                        if words_following > 0:
                            instances.append((line_index,word_index))
                    else:
                        instances.append((line_index,word_index))
        return instances
    
    # Returns (line_index, word_index)
    def find_applause_instances(self, exclude_final = False):
        lines = self.get_text_lines()
        instances = []
        for line_index, l in enumerate(lines):
            if '(Applause)' in l:
                word_index = self.get_word_index_of_applause_in_line(l)
                if exclude_final:
                    words_following = self.count_words_following(line_index,word_index)
                    if words_following > 0:
                        instances.append((line_index,word_index))
                else:
                    instances.append((line_index,word_index))
        return instances
        

In [14]:
transcript = Transcript(transcript_files[1])

In [15]:
instances = transcript.find_applause_instances()
instances

[(2, 0), (4, 0), (20, 0), (22, 0), (26, 0), (64, 0)]

In [16]:
transcript.get_text_lines()

[u'(Whistling)',
 u'(Whistling ends)',
 u'(Applause)',
 u'Thank you.',
 u'(Applause)',
 u"Thank you very much. That was whistling. I'm trying to do this in English. What is a chubby, curly-haired guy from Holland \u2014 why is he whistling? Well actually, I've been whistling since the age of four, about four. My dad was always whistling around the house, and I just thought that's part of communication in my family. So I whistled along with him. And actually, until I was 34, I always annoyed and irritated people with whistling, because, to be honest, my whistling is a kind of deviant behavior. I whistled alone, I whistled in the classroom, I whistled on bike, I whistled everywhere.",
 u"And I also whistled at a Christmas Eve party with my family-in-law. And they had some, in my opinion, terrible Christmas music. And when I hear music that I don't like, I try to make it better.",
 u'(Laughter)',
 u'So when "Rudolph the Red-Nosed Reindeer" \u2014 you know it?',
 u'(Whistling)',
 u'But it 

In [568]:
pre_applause_text = []
for f in transcript_files:
    t = Transcript(f)
    applause_instances = t.find_applause_instances(exclude_final=True)
    for instance in applause_instances:
        text = t.get_preceding_line(instance[0],instance[1])
        pre_applause_text.append(text)

In [569]:
len(pre_applause_text)

2968

In [560]:
glove_path = "/data/jrgillick/word-vectors/glove.840B.300d.txt"

In [561]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [562]:
word_vecs = loadGloveModel(glove_path)

Loading Glove Model


KeyboardInterrupt: 